# Single Microburst Artifitial Neural Network (ANN)
To identify single microbursts in the SAMPEX 20 ms data. The training data comes from the microbursts identified with the O'Brien et al.'s, 2003 burst parameter.

In [1]:
import pathlib

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import microburst_ann.config as config

Specify train and test data paths

In [2]:
train_path = pathlib.Path(config.PROJECT_DIR, 'data', 'train.csv')
test_path = pathlib.Path(config.PROJECT_DIR, 'data', 'test.csv')

## Load data

In [3]:
# Load the csv files
train_df = pd.read_csv(train_path, index_col=0)
test_df = pd.read_csv(test_path, index_col=0)
# Drop the NaN rows
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
# Pop off the training and test labels and make them into their own pd.DataFrames
train_labels = train_df.pop('label')
test_labels = test_df.pop('label')

## pd.DataFrame -> tf.Dataset
This streamlines the data input into Tensorflow and Karas. Now sure what else this is good for.

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_df.to_numpy(), train_labels.to_numpy())
    )
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_df.to_numpy(), test_labels.to_numpy())
    )
shuffled_train_dataset = train_dataset.shuffle(train_df.shape[0]).batch(1)

In [4]:
def create_ann_model():
    """ 
    Specify the ANN model architecture and compile it.
    """
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(50,)),
        tf.keras.layers.Dense(25, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

In [7]:
model = create_ann_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                260       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 1,546
Trainable params: 1,546
Non-trainable params: 0
_________________________________________________________________


So we have 1546 parameters to train. And now lets set up the checkpoint directory to save the model parameters to a binary (h5) file.

In [8]:
checkpoint_path = config.PROJECT_DIR / 'ann' / 'model' / 'model.cp'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
    )

In [ ]:
history = model.fit(shuffled_train_dataset, # Normally, you would indicate the data and labels. However, the tf.Dataset takes care of the input.
                    validation_data=(test_df.to_numpy(), test_labels), # Test dataset used to calculate the model accuracy.
                    epochs=3, # How many epochs to use (train using the entire train dataset epoch times.)
                    callbacks=[checkpoint_callback]) # Set up the callback to save the model.

Now to practice the checkpoint API, lets load the trained weights into a new model and set its weights

In [ ]:
model2 = create_model()
model2.load_weights(checkpoint_path)

## Evaluate the model

The model.evaluate line below is the same as this handwritten loop:

In [ ]:
# n_correct=0
# for i in range(test_df.shape[0]):
#     if round(model(test_df.iloc[i].to_numpy().reshape((1, 50))).numpy()[0][0]) == test_labels.iloc[i]:
#         n_correct += 1
# print(f'n_correct={n_correct}, n_correct/n_total={n_correct/test_df.shape[0]}')

In [ ]:
model.evaluate(test_df.to_numpy(), test_labels, verbose=2)

But model.evaluate is optimized so it is MUCH quicker.

## Plot the model accuracy and loss functions 
as a function of Epoch

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.set_ylim(0, 1) # set the vertical range to [0-1]